In [87]:
from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
import csv
from tqdm.auto import tqdm
tqdm.pandas()

### Read Data and remove problematic strings

In [200]:
normal = pd.read_csv("sentence-aligned.v2/normal.aligned",sep="\t",header=None)
simplified =  pd.read_csv("sentence-aligned.v2/simple.aligned",sep="\t",header=None)
simplified[3] = 1
normal[3] = 1
### For some reason, the tagger doesn't handle -LBR- (left parenthasis "(") and -RRB- (right parenthsis ")") tokenized tag so switched back to "(" and ")"
simplified[2] = simplified[2].str.replace("-LRB-", "(").str.replace("-RRB-", ')').str.replace('-',' ').str.replace("  ", " ").str.replace("  ", " ")
normal[2] = normal[2].str.replace("-LRB-", "(").str.replace("-RRB-", ')').str.replace('-',' ').str.replace("  ", " ").str.replace("  ", " ")

### Tagging function. Takes a sentece and tags it using Stanford tagged (Stanza)

In [109]:
nlp = stanza.Pipeline('en',processors='tokenize,pos')

def tag_sentence_stanza(sentence_str):
    doc = nlp(sentence_str)
    return ' '.join([word.xpos for sent in doc.sentences for word in sent.words])

INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


### Taggs the data

In [217]:
FROM_SENTENCE = 0
TO_SENTENCE = 10000

normal[3][FROM_SENTENCE:TO_SENTENCE] = normal[2][FROM_SENTENCE:TO_SENTENCE].progress_apply(tag_sentence_stanza)
simplified[3][FROM_SENTENCE:TO_SENTENCE] = simplified[2][FROM_SENTENCE:TO_SENTENCE].progress_apply(tag_sentence_stanza)

<ipython-input-217-8d9791befee8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal[3][FROM_SENTENCE:TO_SENTENCE] = normal[2][FROM_SENTENCE:TO_SENTENCE].progress_apply(tag_sentence_stanza)


<ipython-input-217-8d9791befee8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified[3][FROM_SENTENCE:TO_SENTENCE] = simplified[2][FROM_SENTENCE:TO_SENTENCE].progress_apply(tag_sentence_stanza)


### Takes only sentences with matching tag length to sentence length as there are some edge cases where this isn't true (less then 10%)

In [218]:
good_length_of_sentences = ((normal[2].str.count(' ') == normal[3].str.count(' ')) & (simplified[2].str.count(' ') == simplified[3].str.count(' ')))[FROM_SENTENCE:TO_SENTENCE]


normal[2][FROM_SENTENCE:TO_SENTENCE][good_length_of_sentences].to_csv('normal_sentences.txt', sep='\n',index=False, header=False)
simplified[2][FROM_SENTENCE:TO_SENTENCE][good_length_of_sentences].to_csv('simplified_sentences.txt', sep='\n',index=False, header=False)

normal[3][FROM_SENTENCE:TO_SENTENCE][good_length_of_sentences].to_csv('normal_sentences_pos_tags.txt', sep='\n',index=False, header=False)
simplified[3][FROM_SENTENCE:TO_SENTENCE][good_length_of_sentences].to_csv('simplified_sentences_pos_tags.txt', sep='\n',index=False, header=False)
